In [1]:
from models import VCLModel, VanillaMLP
from models.coreset import RandomCoreset
from datasets import PermutedMnist

import matplotlib.pyplot as plt
import numpy as np

import os

import torch
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

In [14]:
NUM_EPOCHS = 100
LR = 1e-3
TRAIN_NUM_SAMPLES = 10
TEST_NUM_SAMPLES = 100
CORESET_SIZE = 200
BATCH_SIZE = 256
INIT_MLP = False

MLP_INIT_DESC = 'withoutmlpinit' if not INIT_MLP else 'withmlpinit'

random_seed = 1
torch.manual_seed(random_seed + 1)
np.random.seed(random_seed)

model_base_name = 'vcl_lr_{}_{}_batch_{}_coresetsize_{}_epochs_{}_run_{}_no_tasks_{}_no_train_samples_{}'

In [15]:
no_tasks = 10

dataset = PermutedMnist(no_tasks)

In [ ]:
NO_RUNS = 3


no_tasks = dataset.no_tasks

# Modify as need be.
lrs = [1e-2, 1e-4]
train_samples = [50, 100]

epochs = [50]
coreset_sizes = [0]

for train_sample in train_samples:
    TRAIN_NUM_SAMPLES = train_sample
    train_loaders, test_loaders = dataset.get_tasks(batch_size=BATCH_SIZE)
    for epoch in epochs:
        NUM_EPOCHS = epoch
        for coreset_size in coreset_sizes:
            CORESET_SIZE = coreset_size
            for run_number in range(NO_RUNS):
                model_name = model_base_name.format(LR, MLP_INIT_DESC, BATCH_SIZE, CORESET_SIZE, NUM_EPOCHS, run_number, no_tasks, TRAIN_NUM_SAMPLES) 
               
                LOG_DIR = 'logs/'
                os.makedirs(LOG_DIR, exist_ok=True)
                
                LOG_FILE_NAME = os.path.join(LOG_DIR, model_name) + '.txt'
                log_file_handler = open(LOG_FILE_NAME, "w")
                print("Run Number: {}. Log File: {}".format(run_number, LOG_FILE_NAME))
                
                if CORESET_SIZE > 0:
                    random_coreset_selector = RandomCoreset(dataset)
                    coreset_loaders, sans_coreset_loaders = random_coreset_selector.get_coreset_loaders(batch_size=BATCH_SIZE, coreset_size=CORESET_SIZE)    
                    
                    # On each task, first train on non-coreset data. 
                    # Else, don't modify train_loaders (train on all the training data).
                    train_loaders = sans_coreset_loaders
                
                model = VCLModel(784, 10, [100, 100])
                accuracies = np.zeros((no_tasks, no_tasks))
                
                for i in range(no_tasks):
                    train_loader = train_loaders[i]
                    
                    if INIT_MLP and i == 0:
                        print("Training MLP model to init first task")
                        mlp = VanillaMLP(784, 10, [100, 100])
                        mlp.train_model(NUM_EPOCHS, train_loader, LR)
                        model.init_mle(mlp)
                    
                    model.train_model(NUM_EPOCHS, train_loader, LR, TRAIN_NUM_SAMPLES)
                    
                    model.update_priors()
                    
                    # Train on coreset after training on non-coreset.
                    if CORESET_SIZE > 0:
                        coreset_loader = coreset_loaders[i]
                        model.train_model(NUM_EPOCHS, coreset_loader, LR, TRAIN_NUM_SAMPLES)
                        
                    task_accs = []
                    for j in range(i + 1):
                        test_loader = test_loaders[j]
                        accuracy = model.get_accuracy(test_loader, 100)
                        
                        msg = "[Number Tasks: {}]: Task {} Accuracy: {}".format(j + 1, j + 1, accuracy ) 
                        log_file_handler.write(msg + '\n')
                        log_file_handler.flush()
                        print(msg)
                        
                        task_accs.append(accuracy)
                        accuracies[i][j] = accuracy
                    
                    msg = "Task {} / {}. Mean Accuracy: {}".format(i + 1, no_tasks, np.mean(task_accs))
                    log_file_handler.write(msg + '\n')
                    log_file_handler.flush()
                    print(msg)
                log_file_handler.close()

Run Number: 0. Log File: logs/vcl_lr_0.001_withoutmlpinit_batch_256_coresetsize_0_epochs_50_run_0_no_tasks_10_no_train_samples_50.txt


Total Loss: 0.48908327691098474, KL: 0.3679376728991245, Lik Loss: 0.1211456041703833: 100%|██████████| 50/50 [03:34<00:00,  4.30s/it] 


[Number Tasks: 1]: Task 1 Accuracy: 0.9788
Task 1 / 10. Mean Accuracy: 0.9788


Total Loss: 0.2880435473741369, KL: 0.12966347279700827, Lik Loss: 0.1583800754331528: 100%|██████████| 50/50 [03:42<00:00,  4.46s/it]  


[Number Tasks: 1]: Task 1 Accuracy: 0.9642
[Number Tasks: 2]: Task 2 Accuracy: 0.9709
Task 2 / 10. Mean Accuracy: 0.9675499999999999


...

Total Loss: 0.2843289720885297, KL: 0.12309240267631856, Lik Loss: 0.1612365702365307: 100%|██████████| 50/50 [11:10<00:00, 13.40s/it]  


[Number Tasks: 1]: Task 1 Accuracy: 0.9182
[Number Tasks: 2]: Task 2 Accuracy: 0.9549
[Number Tasks: 3]: Task 3 Accuracy: 0.9653
Task 3 / 10. Mean Accuracy: 0.9461333333333334


Total Loss: 0.29100911623619974, KL: 0.123182339110273, Lik Loss: 0.16782677814047386: 100%|██████████| 50/50 [10:34<00:00, 12.70s/it]  


[Number Tasks: 1]: Task 1 Accuracy: 0.9013
[Number Tasks: 2]: Task 2 Accuracy: 0.9486
[Number Tasks: 3]: Task 3 Accuracy: 0.9542
[Number Tasks: 4]: Task 4 Accuracy: 0.9645
Task 4 / 10. Mean Accuracy: 0.94215


Total Loss: 0.3075224941715281, KL: 0.11335781834861065, Lik Loss: 0.19416467610825883:  48%|████▊     | 24/50 [05:58<06:02, 13.93s/it] 

In [ ]:
TRAIN_NUM_SAMPLES = 10
it = iter(task_loaders[3][1])

In [85]:
correct_entropy = 0
incorrect_entropy = 0

correct_prob_std = 0
incorrect_prob_std = 0 

incorrect_count, correct_count = 0, 0

model.eval()

for inputs, targets in train_loaders[3]:
    with torch.no_grad():
        inputs, targets = inputs.cuda(), targets.cuda() 
        batch = inputs.size(0)
        
        inputs = torch.tile(inputs, (TEST_NUM_SAMPLES, 1, 1))
        samples = model(inputs)
        
        samples = samples.view(TEST_NUM_SAMPLES, batch, -1)
        samples_mean = torch.mean(samples, dim=0)
        
        samples_prob = F.softmax(samples, dim=-1)
        samples_prob_mean = torch.mean(samples_prob, dim=0)
    
        samples_prob_std = torch.std(samples_prob, dim=0)
        samples_entropy = (-samples_prob_mean * torch.log2(samples_prob_mean + 1e-9)).sum(dim=1)
        
        preds = samples_mean.argmax(dim=-1)
        
        incorrect_indices = (preds != targets).nonzero()
        correct_indices = (preds == targets).nonzero()
        
        incorrect_prob_std += torch.sum(torch.mean(samples_prob_std[incorrect_indices], dim=-1))
        correct_prob_std += torch.sum(torch.mean(samples_prob_std[correct_indices], dim=-1)) 
        
        incorrect_entropy += torch.sum(samples_entropy[incorrect_indices])
        correct_entropy += torch.sum(samples_entropy[incorrect_indices])
        
        incorrect_count += incorrect_indices.count_nonzero()
        correct_count += correct_indices.count_nonzero()
 
incorrect_entropy /= incorrect_count
incorrect_prob_std /= incorrect_count

correct_entropy /= correct_count
correct_prob_std /= correct_count

tensor([7, 9, 2, 3, 1, 5, 2, 1, 6, 5, 1, 4, 3, 6, 3, 0, 9, 2, 7, 2, 9, 9, 0, 5,
        8, 2, 1, 2, 3, 0, 2, 8, 5, 8, 0, 4, 4, 5, 2, 3, 8, 6, 1, 6, 6, 5, 6, 9,
        7, 0, 7, 5, 6, 9, 1, 7, 7, 0, 5, 3, 5, 9, 9, 4, 1, 8, 0, 5, 0, 4, 4, 2,
        5, 3, 2, 9, 1, 0, 8, 9, 0, 8, 5, 2, 8, 0, 6, 4, 9, 8, 8, 9, 9, 3, 3, 2,
        8, 6, 3, 1, 5, 6, 5, 8, 5, 0, 2, 9, 9, 9, 4, 9, 6, 8, 1, 1, 7, 1, 7, 5,
        2, 6, 6, 3, 3, 9, 5, 6, 3, 1, 9, 7, 5, 6, 7, 8, 4, 0, 3, 2, 4, 1, 5, 4,
        3, 5, 5, 1, 5, 1, 4, 9, 4, 9, 8, 0, 4, 6, 7, 1, 5, 7, 2, 2, 1, 9, 5, 0,
        0, 1, 8, 3, 8, 1, 0, 7, 7, 5, 6, 8, 8, 1, 3, 2, 5, 5, 4, 2, 1, 3, 7, 6,
        5, 2, 4, 8, 6, 3, 5, 7, 4, 6, 8, 8, 6, 7, 7, 9, 5, 0, 1, 5, 1, 6, 4, 3,
        2, 4, 0, 2, 3, 8, 1, 4, 1, 0, 3, 3, 2, 9, 8, 8, 4, 1, 7, 6, 3, 8, 3, 0,
        6, 2, 3, 6, 2, 7, 5, 9, 2, 6, 8, 7, 9, 8, 6, 6])


ZeroDivisionError: division by zero

In [66]:
from models.coreset import UncertaintyCoreset

In [115]:
uc = UncertaintyCoreset(dataset, 100)

In [116]:
train_loaders = uc.get_train_loaders(256)

In [117]:
uc_loader = uc.update_uncertainty_coreset_and_get_loader(task_idx=0, 
                                          batch_size=256,
                                          training_task_loader=train_loaders[0],
                                          model=model,
                                          uncertainty_coreset_size=100,
                                          no_samples=100)

In [118]:
correct_entropy = 0
incorrect_entropy = 0

correct_prob_std = 0
incorrect_prob_std = 0 

incorrect_count, correct_count = 0, 0

model.eval()

for inputs, targets in uc_loader:
    with torch.no_grad():
        print(targets)
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        inputs = torch.tile(inputs, (TEST_NUM_SAMPLES, 1, 1))
        samples = model(inputs)
        
        samples_mean = torch.mean(samples, dim=0)
        
        samples_prob = F.softmax(samples, dim=-1)
        samples_prob_mean = torch.mean(samples_prob, dim=0)
    
        samples_prob_std = torch.std(samples_prob, dim=0)
        samples_entropy = (-samples_prob_mean * torch.log2(samples_prob_mean + 1e-9)).sum(dim=1)
        
        preds = samples_mean.argmax(dim=-1)
        
        incorrect_indices = (preds != targets).nonzero()
        correct_indices = (preds == targets).nonzero()
        
        incorrect_prob_std += torch.sum(torch.mean(samples_prob_std[incorrect_indices], dim=-1))
        correct_prob_std += torch.sum(torch.mean(samples_prob_std[correct_indices], dim=-1)) 
        
        incorrect_entropy += torch.sum(samples_entropy[incorrect_indices])
        correct_entropy += torch.sum(samples_entropy[incorrect_indices])
        
        incorrect_count += incorrect_indices.size(0)
        correct_count += correct_indices.size(0)

print(correct_count, incorrect_count)

incorrect_entropy /= incorrect_count
incorrect_prob_std /= incorrect_count

correct_entropy /= correct_count
correct_prob_std /= correct_count

tensor([1, 0, 1, 2, 9, 2, 1, 3, 7, 1, 6, 0, 0, 1, 0, 8, 1, 7, 7, 0, 7, 1, 6, 1,
        7, 1, 1, 1, 9, 4, 1, 1, 1, 5, 0, 1, 8, 2, 7, 1, 2, 1, 2, 6, 2, 3, 1, 8,
        1, 3, 7, 7, 0, 0, 0, 1, 2, 0, 8, 6, 9, 2, 6, 2, 3, 3, 1, 6, 1, 2, 1, 4,
        7, 7, 1, 1, 9, 7, 6, 1, 4, 6, 5, 7, 1, 6, 1, 5, 3, 7, 1, 4, 1, 3, 1, 1,
        2, 7, 1, 2, 0, 1, 6, 1, 0, 5, 1, 5, 9, 7, 5, 1, 3, 1, 1, 2, 1, 0, 7, 1,
        7, 8, 6, 5, 1, 1, 3, 8, 2, 6, 1, 8, 7, 2, 9, 2, 7, 7, 7, 2, 7, 1, 3, 1,
        9, 1, 4, 7, 1, 8, 1, 4, 0, 1, 1, 4, 1, 2, 1, 1, 0, 1, 1, 7, 2, 2, 7, 7,
        1, 6, 7, 0, 5, 1, 3, 1, 1, 1, 5, 1, 0, 4, 5, 2, 7, 1, 1, 1, 0, 9, 7, 5,
        7, 1, 1, 0, 4, 7, 2, 3])
155 45


In [119]:
correct_entropy

tensor(0.7789, device='cuda:0')

In [120]:
samples_prob_std.shape

torch.Size([200, 10])

In [121]:
incorrect_entropy, correct_entropy

(tensor(2.6830, device='cuda:0'), tensor(0.7789, device='cuda:0'))

In [122]:
incorrect_count, correct_count

(45, 155)

In [123]:
incorrect_prob_std, correct_prob_std

(tensor(0.0880, device='cuda:0'), tensor(0.0258, device='cuda:0'))

In [ ]:
accuracies

In [ ]:
for p in model.module_list[0].parameters():
    print(p)

In [ ]:
model.module_list[0].prior_weight_mu